In [3]:
!pip3 install auto-sklearn

     |████████████████████████████████| 6.3 MB 3.8 MB/s 
     |████████████████████████████████| 28.5 MB 52 kB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 722 kB 54.4 MB/s 
     |████████████████████████████████| 4.2 MB 21.8 MB/s 
     |████████████████████████████████| 4.0 MB 39.4 MB/s 
     |████████████████████████████████| 206 kB 71.2 MB/s 
     |████████████████████████████████| 973 kB 67.9 MB/s 
     |████████████████████████████████| 118 kB 65.0 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.13.0-py3-none-any.whl size=6567738 sha256=25caf9bf7090ce7202fde351c569cc6018b25da5d4b38826bc2cb253bde41355
  Stored in directory: /root/.cache/pip/wheels/e8/42/a0/2382eb3f1fdf1c90da8a5302c37f07151128ec64f24c482616
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7043 sha256=d9dcaa2da2d81d21f75b1d2aab8b8c9348d63d69da5383fc39241396344171d1
  Stored in directory: /root/.cache/pip/wheels/42/71/95

In [1]:
%matplotlib inline


# Metrics

In *Auto-sklearn*, model is optimized over a metric, either built-in or
custom metric. Moreover, it is also possible to calculate multiple metrics
per run. The following examples show how to calculate metrics built-in
and self-defined metrics for a classification problem.


In [2]:
from autosklearn.experimental.askl2 import AutoSklearn2Classifier

In [3]:
import autosklearn.classification
import numpy as np
import pandas as pd
import sklearn.datasets
import sklearn.metrics
from autosklearn.metrics import balanced_accuracy, precision, recall, f1


def error(solution, prediction):
    # custom function defining error
    return np.mean(solution != prediction)


def get_metric_result(cv_results):
    results = pd.DataFrame.from_dict(cv_results)
    results = results[results['status'] == "Success"]
    cols = ['rank_test_scores', 'param_classifier:__choice__', 'mean_test_score']
    cols.extend([key for key in cv_results.keys() if key.startswith('metric_')])
    return results[cols]

## Data Loading



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#import all data
data = pd.read_csv(r'/content/drive/MyDrive/msc-project-data/features_with_outcome.csv',encoding='utf=8')

y_data = data['outcome']
X_data = data.drop(columns = ['org_uuid','outcome'])

#splitting data into test and full training set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data,y_data, test_size = 0.2,stratify=y_data)

## Build and fit a classifier



In [ ]:
error_rate = autosklearn.metrics.make_scorer(
    name='custom_error',
    score_func=error,
    optimum=0,
    greater_is_better=False,
    needs_proba=False,
    needs_threshold=False
)
cls = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=120,
    per_run_time_limit=30,
    scoring_functions=[balanced_accuracy, precision, recall, f1, error_rate]
)
cls.fit(X_train, y_train, X_test, y_test)

## Get the Score of the final ensemble



In [ ]:
predictions = cls.predict(X_test)
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, predictions))

print("#" * 80)
print("Metric results")
print(get_metric_result(cls.cv_results_).to_string(index=False))